<a href="https://colab.research.google.com/github/Latindude101/Quantitative-Economics/blob/main/QE_PS6_Q6_(R).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and OLS

In [3]:
install.packages('estimatr')
install.packages('car')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Formula’, ‘RcppEigen’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘matrixStats’, ‘RcppArmadillo’, ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘sp’, ‘data.table’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘statmod’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’




In [5]:
fert <- read.csv('fertility.csv')
head(fert)
library(estimatr)
library(car)

,morekids,boy1st,boy2nd,samesex,agem1,black,hispan,othrace,weeksm1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,1,0,0,27,0,0,0,0
2,0,0,1,0,30,0,0,0,30
3,0,1,0,0,27,0,0,0,0
4,0,1,0,0,35,1,0,0,0
5,0,0,0,1,30,0,0,0,22
6,0,1,0,0,26,0,0,0,40


In [6]:
# Extract values
morekids <- fert$morekids
boy1st <- fert$boy1st
boy2nd <- fert$boy2nd
samesex <- fert$samesex
agem1 <- fert$agem1
black <- fert$black
hispan <- fert$hispan
othrace <- fert$othrace
weeksm1 <- fert$weeksm1


In [8]:
# a)
# OLS regression: Y = B0 + B1Di + U
# morekids = 1 if mum has more than 2 kids
add_kids <- summary(lm_robust(formula = weeksm1 ~ morekids, data = fert))
add_kids
ad_coef <- add_kids$coefficient["morekids","Estimate"]
ad_err <- add_kids$coefficient["morekids","Std. Error"]
ad_err
# Using sprintf() for string interpolation
sprintf("On average, in the sample, the effect of having an additional child when a woman has already had two children on weeks spent working is %s weeks.", round(ad_coef, digits = 1))


Call:
lm_robust(formula = weeksm1 ~ morekids, data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.120    0.04809  439.15        0   21.025   21.214 347759
morekids      -5.407    0.07455  -72.53        0   -5.553   -5.261 347759

Multiple R-squared:  0.01443 ,	Adjusted R-squared:  0.01443 
F-statistic:  5261 on 1 and 347759 DF,  p-value: < 2.2e-16

[1] 0.07454751

[1] "On average, in the sample, the effect of having an additional child when a woman has already had two children on weeks spent working is -5.4 weeks."

In [ ]:
# b)
"The regression in a) will not give an estimate of the causal effect of fertility on labour supply because the residual of the regression is not orthogonal to variables used to estimate the labour supply."
"For example, there is likely to be a correlation between number of children and wealth (not included as a variable in the regression) which also affects number of weeks worked."
"Or women who want to focus on their career may have fewer children (unobserved preferences)."
"Perhaps reverse causality."

[1] "The regression in a) will not give an estimate of the causal effect of fertility on labour supply because the residual of the regression is not orthogonal to variables used to estimate the labour supply."

[1] "For example, there is likely to be a correlation between number of children and wealth (not included as a variable in the regression) which also affects number of weeks worked."

[1] "Or women who want to focus on their career may have fewer children (unobserved preferences)."

[1] "Perhaps reverse causality."

In [ ]:
# c)
# i) t test of mean morekids between boy1st==1 and boy1st==0
m_morekids <- fert$morekids[fert$boy1st==1]
f_morekids <- fert$morekids[fert$boy1st==0]
mean(m_morekids)
mean(f_morekids)

# 1. Hypotheses:
"H0: No diff in mean fertility between those whose first child was male vs female"
"ha: There is a diff in mean fertility between the groups"

# 2. Decision Rule:
"alpha = 0.05"
crit <- qt(0.975, nrow(fert))
sprintf("Reject H0 if |t| > %s", qt(0.975, nrow(fert)))

# 3. Calculate t value:
ttest <- t.test(f_morekids, m_morekids, mu=0, alternative="two.sided")
ttest

# 4. Compare with critical value:
sprintf("%s > %s", ttest$statistic, crit)
"We can therefore reject the hypothesis that there is no difference in fertility between those who have male first child and those who have a female first child."
'Those who have a male first child on average in the same go on to have more than two children less frequently than those who have a female first child.'
"This could be because of the desire to have a son that could mean that those who have a male first child are less likely to have more than two kids that those whose first two children are female."

[1] 0.376275

[1] 0.3851055

[1] "H0: No diff in mean fertility between those whose first child was male vs female"

[1] "ha: There is a diff in mean fertility between the groups"

[1] "alpha = 0.05"

...


	Welch Two Sample t-test

data:  f_morekids and m_morekids
t = 4.5867, df = 253679, p-value = 4.505e-06
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.005057134 0.012603982
sample estimates:
mean of x mean of y 
0.3851055 0.3762750 


[1] "4.5867253687037 > 1.95997330024809"

[1] "We can therefore reject the hypothesis that there is no difference in fertility between those who have male first child and those who have a female first child."

[1] "Those who have a male first child on average in the same go on to have more than two children less frequently than those who have a female first child."

[1] "This could be because of the desire to have a son that could mean that those who have a male first child are less likely to have more than two kids that those whose first two children are female."

In [ ]:
# c)  i) alternatively different effects can be estimated using regressions and testing whether the coefficient is zero
lm_robust(morekids ~ boy1st, data = fert)
# gives same t value as mean comparison
"Estimate of coefficient on boy1st of -0.0088 is statistically significant but very small ('significant' does not mean large in statistics: significant means large relative to its imprecision)"

                Estimate  Std. Error    t value     Pr(>|t|)    CI Lower
(Intercept)  0.385105523 0.001383757 278.304355 0.000000e+00  0.38239340
boy1st      -0.008830558 0.001925242  -4.586725 4.504663e-06 -0.01260398
                CI Upper     DF
(Intercept)  0.387817649 254652
boy1st      -0.005057134 254652

In [15]:
# ii) t test of the mean of morekids between samesex==1 and samesex==0
# samesex==1 when first two children are of the same sex
same_morekids <- fert$morekids[fert$samesex==1]
diff_morekids <- fert$morekids[fert$samesex==0]
mean(same_morekids)
mean(diff_morekids)

# 1. Hypotheses:
"H0: No difference in average fertility between women whose first two children are of the same sex and women whose first two children are of different sex"
"Ha: the average fertilities are not equal"

# 2. Decision Rule:
"alpha = 0.05"
"Reject if t value lies outside +/- 1.959973 (critical values with 254614 d.o.f.)"
# calculate critical t values
nrow(fert)

# inputting 0.975 to qt() gives the critical value at the 0.05% s.l. for a two sided test
# qt() is TDist quantile function
qt(.975, nrow(fert))
# WHY IS MY COUNT OF DOF DIFFERENT FROM R'S T TEST (off by 40)

# 3. Calculate:
ttest <- t.test(same_morekids, diff_morekids, mu=0, alternative="two.sided")
ttest

# 4. Compare critical values
" t value: 41.88 > 1.959973"
sprintf("We can therefore reject the null hypothesis that there is no difference between the average fertility of women whose first two children are of the same sex and women whose first two children are of different sex")
"This could also be due to the desire for a male heir that means that those with two female first children (samesex==1) are more likely to have further children."

[1] 0.4167647

[1] 0.3479284

[1] "H0: No difference in average fertility between women whose first two children are of the same sex and women whose first two children are of different sex"

[1] "Ha: the average fertilities are not equal"

[1] "alpha = 0.05"

...

[1] 1.959971


	Welch Two Sample t-test

data:  same_morekids and diff_morekids
t = 41.877, df = 347727, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.06561455 0.07205802
sample estimates:
mean of x mean of y 
0.4167647 0.3479284 


[1] " t value: 41.89 > 1.959973"

[1] "We can therefore reject the null hypothesis that there is no difference between the average fertility of women whose first two children are of the same sex and women whose first two children are of different sex"

[1] "This could also be due to the desire for a male heir that means that those with two female first children (samesex==1) are more likely to have further children."

In [16]:
# c) ii) using the regression method
summary(lm_robust(formula = morekids ~ samesex, data = fert))


Call:
lm_robust(formula = morekids ~ samesex, data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  0.34795   0.001149  302.71        0  0.34570  0.35020 347761
samesex      0.06879   0.001644   41.84        0  0.06557  0.07201 347761

Multiple R-squared:  0.005008 ,	Adjusted R-squared:  0.005005 
F-statistic:  1751 on 1 and 347761 DF,  p-value: < 2.2e-16

In [ ]:
# d)
'samesex might not be a valid instrument for morekids if it does not meet the exogeneity condition'
"Three conditions for a valid instrument: exclusion, relevance, exogeneity"
'samesex is exogenously assigned (assigned as if random) (theoretically) but may not only affect labour supply through morekids:'
"exogeneity could be tested with an F test of the other variables determining samesex"
"c): samesex is revelant to the determination of morekids"
'It could be that having children of different sexes means more work for the mother, e.g. ferrying them to different activities, resulting in correlation between samesex and weeksm1'
"This would be an effect of samesex on labour supply that does not go through morekids"

# Regression d
exog_reg <- lm_robust(formula = weeksm1 ~ samesex, data=fert)

# F test of significance
# hypothesis matrix requires single =

# 1. Hypotheses:
# H0: OLS coefficient on samesex = 0
# Ha: OLS coefficient on samesex != 0

# 2. Decision Rule:
# Reject H0 if F statistic > 10

# 3. Calculate F statistic
ftest <- linearHypothesis(exog_reg, c("samesex=0"), test="F")
ftest

# 4. Compare F statistic to critical value:
sprintf("%s > %s", ftest$F[2], 10)
"Therefore we can reject the null hypothesis"


[1] "samesex might not be a valid instrument for morekids if it does not meet the exogeneity condition"

[1] "samesex is exogenously assigned (assigned as if random) but may not only affect labour supply through morekids:"

[1] "It could be that having children of different sexes means more work for the mother, e.g. ferrying them to different activities, resulting in correlation between samesex and weeksm1"

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,254653,NA,NA,NA
2,254652,1,24.1957,8.708031e-07


[1] "24.1957035172991 > 10"

[1] "Therefore we can reject the null hypothesis"

# 2SLS

In [ ]:
# e)
# 2SLS manually
# Model: y = B0 + B1Xi + Ui
# weeksm1 ~ morekids
# Regress morekids on the instrument to calculate fitted values
fsls <- lm_robust(formula = morekids ~ samesex, data = fert)
summary(fsls)
# how are fitted values different from morekids?
morekids_hat = fitted.values(fsls)
tsls <- lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)
summary(tsls)



Call:
lm_robust(formula = morekids ~ samesex, data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value   Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  0.34642   0.001341  258.34  0.000e+00  0.34380  0.34905 254652
samesex      0.06753   0.001919   35.19 1.388e-270  0.06376  0.07129 254652

Multiple R-squared:  0.004835 ,	Adjusted R-squared:  0.004831 
F-statistic:  1238 on 1 and 254652 DF,  p-value: < 2.2e-16


Call:
lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)

Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)    21.421     0.4906  43.663 0.000e+00   20.460   22.383 254652
morekids_hat   -6.314     1.2836  -4.919 8.708e-07   -8.829   -3.798 254652

Multiple R-squared:  9.502e-05 ,	Adjusted R-squared:  9.109e-05 
F-statistic:  24.2 on 1 and 254652 DF,  p-value: 8.708e-07

In [ ]:
# e) using packages
# iv_robust(formula = y_var ~ endog + ctrl1 + ctrl2 | instr1 + instr2 + ctrl1 + ctrl2, data = data, diagnostics = TRUE)
# endogenous variable to be instrumented for on the left of the vertical pipe only
# instruments on the right of the vertical pipe only
# control variables (neither instruments nor endogenous variables to be instrumented for) on both sides of the vertical pipe
ivreg <- iv_robust(formula = weeksm1 ~ morekids | samesex, diagnostics=TRUE)
iv_sum <- summary(ivreg)
iv_sum
sprintf('SE of 2SLS (%s) >> SE of OLS (%s): samesex is not a valid instrument as explained in d)', iv_sum$coefficient["morekids", "Std. Error"], ad_err)
'Estimate of -6.3 from 2SLS vs -5.3 from OLS:'


Call:
iv_robust(formula = weeksm1 ~ morekids | samesex, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.421     0.4873  43.963 0.000e+00   20.466   22.376 254652
morekids      -6.314     1.2747  -4.953 7.308e-07   -8.812   -3.815 254652

Multiple R-squared:  0.01388 ,	Adjusted R-squared:  0.01388 
F-statistic: 24.53 on 1 and 254652 DF,  p-value: 7.308e-07

Diagnostics:
                  numdf  dendf    value p.value    
Weak instruments      1 254652 1238.171  <2e-16 ***
Wu-Hausman            1 254651    0.531   0.466    
Overidentifying       0     NA       NA      NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "SE of 2SLS (1.27468565210554) >> SE of OLS (0.0871491793698437): samesex is not a valid instrument as explained in d)"

[1] "Estimate of -6.3 from 2SLS vs -5.3 from OLS:"

In [20]:
# f) manually
# othrace does not include white so no perfect multicollinearity with black, hispan, and othrace
# missed out age!!!
sum(fert$black)
sum(fert$othrace)
sum(fert$hispan)
fsls <- lm_robust(formula = morekids ~ samesex + black + hispan + othrace, data = fert)
summary(fsls)
# without controls, fitted.values(fsls) would also give morekids_hat; with controls it is shorter in length than the other variables
morekids_hat <- predict(fsls)
length(morekids_hat)
length(othrace)
tsls <- lm_robust(formula = weeksm1 ~ morekids_hat + black + hispan + othrace, data = fert)
summary(tsls)

[1] NA

[1] NA

[1] NA


Call:
lm_robust(formula = morekids ~ samesex + black + hispan + othrace, 
    data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value   Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  0.33123   0.001183 279.949  0.000e+00  0.32891  0.33355 347755
samesex      0.06905   0.001637  42.187  0.000e+00  0.06584  0.07226 347755
black        0.09240   0.003838  24.078 5.438e-128  0.08488  0.09992 347755
hispan       0.14933   0.003616  41.300  0.000e+00  0.14224  0.15642 347755
othrace      0.02390   0.004138   5.776  7.666e-09  0.01579  0.03201 347755

Multiple R-squared:  0.01346 ,	Adjusted R-squared:  0.01345 
F-statistic:  1162 on 4 and 347755 DF,  p-value: < 2.2e-16

[1] 347764

[1] 347764


Call:
lm_robust(formula = weeksm1 ~ morekids_hat + black + hispan + 
    othrace, data = fert)

Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  20.77688     0.3934 52.8079 0.000e+00  20.0057  21.5480 347755
morekids_hat -6.27328     1.0682 -5.8726 4.295e-09  -8.3670  -4.1796 347755
black        11.41072     0.1980 57.6399 0.000e+00  11.0227  11.7987 347755
hispan        0.02556     0.2255  0.1133 9.098e-01  -0.4164   0.4675 347755
othrace       1.71770     0.1891  9.0836 1.056e-19   1.3471   2.0883 347755

Multiple R-squared:  0.01214 ,	Adjusted R-squared:  0.01213 
F-statistic:  1018 on 4 and 347755 DF,  p-value: < 2.2e-16

In [ ]:
# f)
iv_robust(weeksm1 ~ morekids + agem1 + black + hispan + othrace | samesex  + agem1 + black + hispan + othrace, data = fert, diagnostics = TRUE)

              Estimate Std. Error    t value      Pr(>|t|)   CI Lower
(Intercept) -4.7918935 0.38979270 -12.293441  1.004902e-34 -5.5558768
morekids    -5.8210509 1.24640073  -4.670288  3.009303e-06 -8.2639631
agem1        0.8315975 0.02264086  36.729935 1.446269e-294  0.7872220
black       11.6232731 0.23180281  50.142934  0.000000e+00 11.1689458
hispan       0.4041802 0.26080282   1.549754  1.212018e-01 -0.1069863
othrace      2.1309620 0.21099514  10.099579  5.606068e-24  1.7174172
              CI Upper     DF
(Intercept) -4.0279102 254648
morekids    -3.3781388 254648
agem1        0.8759730 254648
black       12.0776004 254648
hispan       0.9153468 254648
othrace      2.5445068 254648

In [21]:
# g)
# perfect multicollinearity of samesex and two_male and two_female?
# Two_male: is the first child a girl? If girl, then 0. If boy, then if samesex, then 1. If boy but not samesex, then 0.
two_male <- ifelse((fert$boy1st==0), 0, ifelse((fert$samesex==1), 1, 0))
two_female <- ifelse((fert$boy1st==1), 0, ifelse((fert$samesex==1), 1, 0))
#two_male
#two_female

# adding/excluding samesex does not change anything
ivreg <- iv_robust(formula = weeksm1 ~ morekids | samesex + black + hispan + othrace + two_male + two_female, diagnostics=TRUE)
summary (ivreg)


ivreg <- iv_robust(formula = weeksm1 ~ morekids | two_male + two_female, diagnostics=TRUE)
summary (ivreg)
'The regression with just two_male and two_female as instruments, unsurprisingly, gives very similar estimates to the regression using samesex as an instrument.'


Call:
iv_robust(formula = weeksm1 ~ morekids | samesex + black + hispan + 
    othrace + two_male + two_female, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)    15.11     0.2675   56.50 0.000e+00   14.590    15.64 347758
morekids       10.27     0.6923   14.83 9.835e-50    8.909    11.62 347758

Multiple R-squared:  -0.1068 ,	Adjusted R-squared:  -0.1068 
F-statistic: 219.9 on 1 and 347758 DF,  p-value: < 2.2e-16

Diagnostics:
                  numdf  dendf value p.value    
Weak instruments      6 347753    NA      NA    
Wu-Hausman            2 347756   301  <2e-16 ***
Overidentifying       5     NA  3026  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Call:
iv_robust(formula = weeksm1 ~ morekids | two_male + two_female, 
    diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.369     0.4019  53.170 0.000e+00   20.581    22.16 347759
morekids      -6.059     1.0450  -5.798 6.723e-09   -8.107    -4.01 347759

Multiple R-squared:  0.01422 ,	Adjusted R-squared:  0.01422 
F-statistic: 33.61 on 1 and 347759 DF,  p-value: 6.723e-09

Diagnostics:
                  numdf  dendf   value p.value    
Weak instruments      2 347758 916.994  <2e-16 ***
Wu-Hausman            1 347758   0.391  0.5320    
Overidentifying       1     NA   3.317  0.0686 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "The regression with just two_male and two_female as instruments, unsurprisingly, gives very similar estimates to the regression using samesex as an instrument."

In [ ]:
# Peter's dummies use logical operations on dummy values converting them to Booleans
two_boys <- as.numeric( fert$samesex & fert$boy1st )
two_girls <- as.numeric( fert$samesex & !fert$boy1st )

In [23]:
# Peter's iv_robust() with diff variables names
summary(iv_robust(weeksm1 ~ morekids + agem1 + black + hispan + othrace | two_male + two_female + agem1 + black + hispan + othrace, data = fert, diagnostics = TRUE))



Call:
iv_robust(formula = weeksm1 ~ morekids + agem1 + black + hispan + 
    othrace | two_male + two_female + agem1 + black + hispan + 
    othrace, data = fert, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  -4.8739    0.32952 -14.791 1.732e-49  -5.5198  -4.2281 347754
morekids     -5.6072    1.02126  -5.490 4.011e-08  -7.6088  -3.6056 347754
agem1         0.8340    0.01873  44.531 0.000e+00   0.7973   0.8707 347754
black        11.8018    0.19660  60.031 0.000e+00  11.4165  12.1871 347754
hispan        0.6123    0.22808   2.685 7.260e-03   0.1653   1.0593 347754
othrace       1.7996    0.18602   9.674 3.910e-22   1.4350   2.1642 347754

Multiple R-squared:  0.0442 ,	Adjusted R-squared:  0.04419 
F-statistic:  1940 on 5 and 347754 DF,  p-value: < 2.2e-16

Diagnostics:
                  numdf  dendf   value p.value    
Weak instruments      2 347753 949.507  <2e-16 ***
Wu-Hausman  

In [ ]:
# h)
'p value of 0.06 > 0.05 in Overidentifying restrictions means that there is not sufficient evidence at the 5% significance level to reject the null of exogeneity for two_male and two_female as instrumental variables'
'You do not want to reject exogeneity'

In [27]:
summary(lm_robust(formula = two_female ~ agem1 + black + hispan + othrace))


Call:
lm_robust(formula = two_female ~ agem1 + black + hispan + othrace)

Standard error type:  HC2 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)  CI Lower   CI Upper     DF
(Intercept)  0.2584244  0.0065907 39.2104 0.000000  0.245507  0.2713420 347755
agem1       -0.0006302  0.0002147 -2.9349 0.003337 -0.001051 -0.0002093 347755
black        0.0020993  0.0033049  0.6352 0.525285 -0.004378  0.0085769 347755
hispan       0.0048124  0.0031266  1.5392 0.123763 -0.001316  0.0109406 347755
othrace     -0.0063889  0.0035841 -1.7826 0.074660 -0.013414  0.0006359 347755

Multiple R-squared:  3.881e-05 ,	Adjusted R-squared:  2.731e-05 
F-statistic: 3.365 on 4 and 347755 DF,  p-value: 0.009234